<a href="https://colab.research.google.com/github/Mia-Tay/collab_housing/blob/main/housing_price_part1_postcode_neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder= 'drive/My Drive/learning/'

Mounted at /content/drive


In [ ]:
import pandas as pd
import requests
import time


In [ ]:
df = pd.read_csv(folder+'Geospatial_data.csv').copy()
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
len(df)

103

In [ ]:
#Call city API and find intersection's corresponding neighborhood

for i in range(len(df)):
    time.sleep(0.3)

    lan = df.iloc[i]['Latitude']
    lng = df.iloc[i]['Longitude']
    r = requests.get(
        f'https://gis.toronto.ca/arcgis/rest/services/primary/cot_geospatial_webm/MapServer/265/query?where=&text=&objectIds=&time=&geometry={lng}%2C{lan}&geometryType=esriGeometryPoint&inSR=4326&spatialRel=esriSpatialRelWithin&relationParam=&outFields=AREA_ID%2CAREA_SHORT_CODE%2CAREA_NAME&returnGeometry=false&maxAllowableOffset=&geometryPrecision=&outSR=3857&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&returnTrueCurves=false&resultOffset=&resultRecordCount=&f=pjson')

    retry_counter = 5
    # if code not 200 (4xx or 5xx) -try again, if content almost empty -try again
    while (len(r.content) < 5 and retry_counter > 0) or (r.status_code != 200 and retry_counter > 0):
        print("HIT ERROR, RETRYING", retry_counter)
        time.sleep(0.3)
        retry_counter = retry_counter - 1
        r = requests.get(
            f'https://gis.toronto.ca/arcgis/rest/services/primary/cot_geospatial_webm/MapServer/265/query?where=&text=&objectIds=&time=&geometry={lng}%2C{lan}&geometryType=esriGeometryPoint&inSR=4326&spatialRel=esriSpatialRelWithin&relationParam=&outFields=AREA_ID%2CAREA_SHORT_CODE%2CAREA_NAME&returnGeometry=false&maxAllowableOffset=&geometryPrecision=&outSR=3857&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&returnTrueCurves=false&resultOffset=&resultRecordCount=&f=pjson')

    if (len(r.content) < 5 or r.status_code != 200):
        print('SORRY, NOTHING WORKS')
        continue
    result = r.json()
    print(result)
    if len(result['features']) > 0:
        area_name = result['features'][0]['attributes'].get('AREA_NAME')
    else:
        area_name = "Not Found"
    df.loc[i, 'hoods'] = area_name
    print('i', i, 'area_name', area_name, 'content', result)
df.to_csv(folder+'postcode_neighborhoods.csv')


In [ ]:
df.to_csv(folder+'postcode_neighborhoods.csv')